In [ ]:
!python gen_images.py --help

In [ ]:
!python gen_video.py --help

In [ ]:
import os
import re
from typing import List, Optional, Tuple, Union

import click
import dnnlib
import numpy as np
import PIL.Image
import torch

import legacy

pklPath='C:/Users/hello/Desktop/AI/lucid-sonic-dreams/pytorch/'
outDir='C:/Users/hello/Desktop/AI/stylegan3/out/'
seeds=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,17,18,19,20]
class_idx=0
truncation_psi=1
noise_mode='const'


pklList = os.listdir(pklPath)

for pkl in pklList:
    pklName = os.path.splitext(pkl)[0]
    network_pkl = pklPath + pkl
    outputdir = outDir + pklName
    vid = outputdir + '/video.mp4'
    if not os.path.exists(outputdir):
        os.makedirs(outputdir)
    print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

    os.makedirs(outputdir, exist_ok=True)

    # Labels.
    label = torch.zeros([1, G.c_dim], device=device)
    if G.c_dim != 0:
        if class_idx is None:
            raise click.ClickException('Must specify class label with --class when using a conditional network')
        label[:, class_idx] = 1
    else:
        if class_idx is not None:
            print ('warn: --class=lbl ignored when running on an unconditional network')

    # Generate images.
    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)

        # Construct an inverse rotation/translation matrix and pass to the generator.  The
        # generator expects this matrix as an inverse to avoid potentially failing numerical
        # operations in the network.
        if hasattr(G.synthesis, 'input'):
            m = make_transform(translate, rotate)
            m = np.linalg.inv(m)
            G.synthesis.input.transform.copy_(torch.from_numpy(m))

        img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outputdir}/seed{seed:04d}.png')
#     !python gen_video.py --network={network} --seeds=1-25 --output={outputdir+'/video.mp4'} --shuffle-seed=20 --grid=1x1 --w-frames=85
    
    
    

C:\Users\hello\anaconda3\envs\lucid\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading networks from "C:/Users/hello/Desktop/AI/lucid-sonic-dreams/pytorch/70s-scifi-1024.pkl"...
warn: --class=lbl ignored when running on an unconditional network
Generating image for seed 1 (0/19) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 